In [79]:
import argparse
import pandas as pd
from tqdm import tqdm
import yaml
import math
import numpy as np
from datetime import datetime
import json
import bisect

config = yaml.load(open('../config/config.yaml'))
nc_info_dict = json.load(open('../config/nc_info.json'))
nc_info_keys = list(nc_info_dict.keys())
exception_name_columns = config['exception_name']['train']

<ipython-input-79-5626503932b7>:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open('../config/config.yaml'))


In [71]:
n_exceptions_train = pd.read_csv('../'+config['data']['features']['train']['n_exceptions_recent_week'])
n_exceptions_test = pd.read_csv('../'+config['data']['features']['test']['n_exceptions_recent_week'])
nc_info_train = pd.read_csv('../'+config['data']['raw']['train']['nc_info'])
nc_info_test = pd.read_csv('../'+config['data']['raw']['test']['nc_info'])


384332

In [74]:
# train
for key in nc_info_keys:
    for val in nc_info_dict[key]:
        nc_info_train[val] = 0
for key in tqdm(nc_info_keys, total=len(nc_info_keys)):
    for i in range(len(nc_info_train)):
        val = nc_info_train.loc[i, key]
        if type(val) == str:
            nc_info_train.loc[i, val] = 1


100%|██████████| 9/9 [00:21<00:00,  2.39s/it]


In [75]:
# test
for key in nc_info_keys:
    for val in nc_info_dict[key]:
        nc_info_test[val] = 0
for key in tqdm(nc_info_keys, total=len(nc_info_keys)):
    for i in range(len(nc_info_test)):
        val = nc_info_test.loc[i, key]
        if type(val) == str:
            nc_info_test.loc[i, val] = 1

100%|██████████| 9/9 [00:05<00:00,  1.80it/s]


In [76]:
nc_info_train_one_hot = nc_info_train[['nc_ip'] + [val for key in nc_info_keys for val in nc_info_dict[key]]]
nc_info_test_one_hot = nc_info_test[['nc_ip'] + [val for key in nc_info_keys for val in nc_info_dict[key]]]

In [77]:
n_exceptions_with_nc_info_train = n_exceptions_train.join(nc_info_train_one_hot.set_index('nc_ip'), on='nc_ip', how='left')
n_exceptions_with_nc_info_test = n_exceptions_test.join(nc_info_test_one_hot.set_index('nc_ip'), on='nc_ip', how='left')


In [80]:
n_exceptions_with_nc_info_train.to_csv('../'+config['data']['features']['train']['n_exceptions_recent_week_with_nc_info'], index=False)
n_exceptions_with_nc_info_test.to_csv('../'+config['data']['features']['test']['n_exceptions_recent_week_with_nc_info'], index=False)

In [68]:
n_exceptions_with_nc_info_test

,nc_ip,sample_time,nc_down_label,exception_name_10_cnt,exception_name_11_cnt,exception_name_12_cnt,exception_name_14_cnt,exception_name_15_cnt,exception_name_16_cnt,exception_name_17_cnt,...,bmc_version_88,bmc_version_9,bmc_version_90,bmc_version_92,bmc_version_93,bmc_version_94,bmc_version_95,bmc_version_96,bmc_version_98,bmc_version_99
0,dccf231b-5b94-4763-9f07-232fd365929e,2017-01-20 15:25:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,7f515357-2f44-4513-a554-de0e06245160,2017-03-28 15:25:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,d2657067-9a4b-49ac-8287-5f14295d903a,2016-04-09 15:25:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,7cd5b33e-a152-481a-95d9-3601a6137622,2020-03-28 15:25:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0bfc7443-2527-4fd5-9669-6acb7643397c,2016-04-23 16:30:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146145,f8553f4b-03a7-4cc6-b6f6-8d52e1eed2f6,2020-02-27 17:40:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
146146,de556185-ee86-410e-9c9d-ed7059650a76,2019-04-21 15:15:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
146147,da9720c1-6797-49ec-8b55-b0d9b9b7394a,2019-12-23 11:00:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146148,fb96a605-1aad-40ed-aeee-c30211bfacb3,2016-02-16 00:45:00,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
